In [1]:
import numpy as np 
import pandas as pd 
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Average
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 256

x_test = []
df_test = pd.read_csv('/input/plant_seedlings_classification/sample_submission.csv')

dim = 256
for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('/input/plant_seedlings_classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)
x_test = x_test /255.0

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
100%|██████████| 794/794 [00:05<00:00, 150.52it/s]


(794, 256, 256, 3)


In [7]:
# Load all models ( Best performing --> Xception and InceptionResNetV2)
base_model = Xception(input_shape=(dim, dim, 3), include_top=False, weights=None, pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model_x = Model(inputs=base_model.input, outputs=predictions)

base_model = InceptionResNetV2(input_shape=(dim, dim, 3), include_top=False, weights=None, pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model_ir = Model(inputs=base_model.input, outputs=predictions)

In [16]:
# Create a model that outputs average prediction
avg = Average()([model_x.output, model_ir.output])
model_ensemble = Model(inputs = [model_x.input,model_ir.input], output = avg)

model_x.load_weights('/input/weights_x.h5')
model_ir.load_weights('/input/weights_ir.h5')

# Function to generate predictions
def predict_ensemble(model):
    p_test = model.predict([x_test,x_test], verbose = 1)
    return p_test

# Get probabilites
p_test = predict_ensemble(model_ensemble)

# Convert probabilites to class names
def get_class(p_test):
    preds = []
    for i in range(len(p_test)):
        pos = np.argmax(p_test[i])
        preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])
    return preds

preds = get_class(p_test)

df_test['species'] = preds
df_test.to_csv('/output/submission.csv', index=False)

/usr/local/share/jupyter/kernels/neptune-kernel/neptunekernel.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("av...)`
  #
